In [1]:
import numpy as np
import pandas as pd

import matplotlib.cm as cm
import matplotlib

from bokeh.io import show, output_notebook, output_file
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper, CustomJS, Slider, CheckboxGroup, Select, Div, TapTool
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
from bokeh.layouts import widgetbox, row
from bokeh.tile_providers import get_provider, Vendors
from bokeh.sampledata.us_counties import data as counties
from bokeh import events
from bokeh.colors.groups import yellow

from pyproj import Proj, transform

from postreise.analyze.transmission import generate_cong_stats
from powersimdata.scenario.scenario import Scenario

In [2]:
output_notebook()

Loading BokehJS ...

### Load Scenario and Retrieve Branches

In [3]:
scenario = Scenario('original')
branch = scenario.state.get_grid().branch.copy()

SCENARIO: base | original

--> State
analyze
--> Loading Western interconnect
Loading zone
Loading sub
Loading bus2sub
Loading bus
Loading plant
Loading plant cost
Loading branch
Loading DC line


### Projection from lon/lat to marcator for openstreetmap

In [4]:
def reproject_wgs_to_itm(x_lon_lat):
    prj_wgs = Proj(init='epsg:4326')
    prj_itm = Proj(init='EPSG:3857')
    x, y = transform(prj_wgs, prj_itm, x_lon_lat[0], x_lon_lat[1])
    r = [x,y]
    return r

In [5]:
r_from = branch[['from_lon','from_lat']].apply(reproject_wgs_to_itm,axis=1)
branch['from_x'] = r_from.apply(lambda x: x[0])
branch['from_y'] = r_from.apply(lambda x: x[1])

r_to = branch[['to_lon','to_lat']].apply(reproject_wgs_to_itm,axis=1)
branch['to_x'] = r_to.apply(lambda x: x[0])
branch['to_y'] = r_to.apply(lambda x: x[1])

### County names and shapes 

In [6]:
state_list = ['wa','or','ca','nv','az','ut','nm','co','wy','id','mt','tx']

# This just shades CA for reference; 
counties = {code: county for code, county in counties.items() if county["state"] == "ca"}

# Convert lon/lat to x/y
county_xys = [reproject_wgs_to_itm([county["lons"],county["lats"]]) for county in counties.values()]

county_xs = [county_xys[i][0] for i,county in enumerate(counties.values())]
county_ys = [county_xys[i][1] for i,county in enumerate(counties.values())]

county_names = [county['name'] for county in counties.values()]
county_color = []
for c_name in county_names:
    color = 'black'
    county_color.append(color) 

source = ColumnDataSource(data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    color=county_color,
))

In [7]:
def makeMap(congestion):
    util_rate = congestion['hutil>=0p75']
    branch_map = pd.concat([branch, congestion], axis=1)

    # Replace this population-derived values from congestion_p-values notebook, or use the zsore from the notebook remove this
    mean = branch_map.loc[branch_map['capacity'] != 99999]['hutil>=0p75'].mean()
    sdev = branch_map.loc[branch_map['capacity'] != 99999]['hutil>=0p75'].std()

    minima = util_rate.min()
    maxima = util_rate.max()

    norm =  matplotlib.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap=cm.jet)
    mapper.set_array([])
    colors = mapper.to_rgba(util_rate.values)

    # Pick colors and linewidths for 3 types of lines depending on zscore = (x-mu)/sigma
    # gray, light red, dark red for increasing congestion
    colors = ['#AAB7BB','#FF5733','#C70039']  
    # thin, thick, thick for increasing congestion
    linewidths = [0.5, 5.0, 5.0]  

    # Replace with pre-calculated z-score
    zscore_t1 = 2.
    zscore_t2 = 1.
    branch_map.loc[(branch_map['zscore'] >= zscore_t1),'color'] = colors[2]
    branch_map.loc[(branch_map['zscore']  < zscore_t1) & (branch_map['zscore']  >= zscore_t2),'color'] = colors[1]
    branch_map.loc[(branch_map['zscore'] < zscore_t2),'color'] = colors[0]

    branch_map.loc[(branch_map['zscore']  >= zscore_t1),'linewidth'] = linewidths[2]
    branch_map.loc[(branch_map['zscore']  < zscore_t1) & (branch_map['zscore']  >= zscore_t2),'linewidth'] = linewidths[1]
    branch_map.loc[(branch_map['zscore']  < zscore_t2),'linewidth'] = linewidths[0]

    # Set up figure
    TOOLS = "pan,wheel_zoom,reset,hover,save"

    p = figure(title="Western Interconnect", tools=TOOLS, x_axis_location=None, y_axis_location=None,
               plot_width=800,plot_height=800)

    p.add_tile(get_provider(Vendors.CARTODBPOSITRON))
    p.patches('x', 'y', source=source, fill_color={'field':'color'}, fill_alpha=0.1, line_color="white",  line_width=0.5)
    p.multi_line(branch_map[['from_x','to_x']].values.tolist(), 
                 branch_map[['from_y','to_y']].values.tolist(), 
                 line_color=branch_map['color'], 
                 line_width=branch_map['linewidth'])
    return p

In [8]:
try:
    congestion = pd.read_csv('../data/congestion_stats_base_demo.csv')
except FileNotFoundError:
    print('Congestion file not found. Will generate it.')
    pf = scenario.state.get_pf()

    branch.loc[branch.rateA != 0, 'capacity'] = branch['rateA']
    branch.loc[branch.rateA == 0, 'capacity'] = 99999.

    pf_norm = pf.div(branch['capacity']).apply(np.abs) 

                                
    congestion = generate_cong_stats(pf_norm, branch,
                                     '../data/congestion_stats_base_demo')
                                
show(makeMap(congestion))